# Определяне на настроения в ревюта от Амазон

## Използвани данни

Набора от данни с Амазон ревюта съдържа 730000 ревюта. Автор на данните в текущият им вид е: `Xiang Zhang (xiang.zhang@nyu.edu)`. Данните са били използвани в научната статия: 
> Xiang Zhang, Junbo Zhao, Yann LeCun. Character-level Convolutional Networks for Text Classification. Advances in Neural Information Processing Systems 28 (NIPS 2015)

Лиценз: няма

Източник: https://drive.google.com/file/d/0Bz8a_Dbh9QhbZVhsUnRWRDhETzA/view?usp=share_link&resourcekey=0-Rp0ynafmZGZ5MflGmvwLGg

## Използвани библиотеки

- NLTK
	* Богата библиотека за обработка на естествен език.
	* Съдържа в себе си лемизатори, стемери и т.н.
- WordNet
	* Богат речник от думи на английски език.
	* Съдържа се информация за POS таговете.
- SentiWordNet
	* Надграждащ речник над WordNet, който оценява емоцията на дадена дума.
	* Съдържа атрибути за позитивна и негативна оценка на дадена дума.

## Включване на библиотеки

In [343]:
import re
import nltk

from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

## Дефиниране на функции за определяне на настроението на дадено изречение

### Определяне на това каква част на речта е дадена дума

`NLTK` разпознава твърде много части на речта. За нашите цели се интересуваме само от:
- прилагателни
- съществителни
- глаголи
- наречия

Това ограничение и преобразуване се налага заради това, че `WordNet` речника разпознава само тези части на речта. Ако дадена дума не можем директно да я приравним към някоя от категориите на `WordNet` - определяме я като съществително.

In [344]:
def get_wordnet_pos(word):
    tag = pos_tag([word])[0][1][0].upper()

    tag_dict = {"J": wn.ADJ,
                "N": wn.NOUN,
                "V": wn.VERB,
                "R": wn.ADV}
    return tag_dict.get(tag, wn.NOUN)


### Определяне на настроението на дадено изречение

- Изплолзваме `word_tokenize` от `NLTK`, за да разделим даденото изречение на символи. За символ тук се възприема цяла дума или препинателен знак.
- Използваме `WordNetLemmatizer` за да намалим броя думи до основното им значение.
- Обхождат се всички символи в даденото изречение.
- Определя се каква част на речта са.
- Намира се лемата.
- Ако лемата се открие в речника на WordNet ѝ се взимат екстремумите на най-близките значения до първото.
- Използвайки `SentiWordNet` речника прибавяме разликата м/у позитивната и отрицателната оценка на думата към общия сбор.
- Нормализираме данните като разделяме общия сбор на оценките на брой емоционални думи.

In [383]:
synset_pattern = re.compile('(.*\.\w)\.\d*')

def get_sentiment_score(sentence):
    # 1. Tokenize
    tokens = word_tokenize(sentence)

    lemmatizer = WordNetLemmatizer()

    sentiment_score = 0.0
    sentiment_word_count = 0
    for word in tokens:
        word_lower = word.lower()
        tag = get_wordnet_pos(word_lower)
        item_res = lemmatizer.lemmatize(word_lower, tag)
        
        synsets = wn.synsets(item_res, pos=tag)

        if len(synsets) == 0:
            continue

        first_synset = synsets[0].name()

        synset_match = synset_pattern.match(first_synset)
        
        if synset_match is None:
            print(f'SS: {first_synset}')

        common_pattern = synset_pattern.match(first_synset).groups()[0]

        max_pos_score = -1
        max_neg_score = -1

        for synset in synsets:
            if synset.name().startswith(common_pattern):
                if max_neg_score < swn.senti_synset(synset.name()).neg_score():
                    max_neg_score = swn.senti_synset(synset.name()).neg_score()
                if max_pos_score < swn.senti_synset(synset.name()).pos_score():
                    max_pos_score = swn.senti_synset(synset.name()).pos_score()

        delta_score = max_pos_score - max_neg_score
        sentiment_score += delta_score

        if delta_score != 0:
            sentiment_word_count += 1
        
    return sentiment_score / sentiment_word_count if sentiment_score != 0 else 0

### Няколко примера:

In [384]:
get_sentiment_score('This is as bad as it gets!')

-0.08333333333333333

In [385]:
get_sentiment_score('Great!')

0.625

In [386]:
get_sentiment_score('This is ok!')

0.125

# Зареждане на набора от данни (корпуса)

In [387]:
import pandas as pd

train = pd.read_csv('./amazon_review_full_csv/train.csv')
test = pd.read_csv('./amazon_review_full_csv/test.csv')

data_columns = ['rating', 'title', 'review']

train.columns = data_columns
test.columns = data_columns

In [388]:
test.head()

,rating,title,review
0,4,Surprisingly delightful,This is a fast read filled with unexpected hum...
1,2,"Works, but not as advertised",I bought one of these chargers..the instructio...
2,2,Oh dear,I was excited to find a book ostensibly about ...
3,2,Incorrect disc!,"I am a big JVC fan, but I do not like this mod..."
4,2,Incorrect Disc,"I love the style of this, but after a couple y..."


In [389]:
train.head()

,rating,title,review
0,5,Inspiring,I hope a lot of people hear this cd. We need m...
1,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
3,5,Too good to be true,Probably the greatest soundtrack in history! U...
4,5,There's a reason for the price,"There's a reason this CD is so expensive, even..."


In [390]:
test_size = test.shape[0]
train_size = train.shape[0]

print(f'Съотношение тренировъчни / тестови = {test_size / (test_size + train_size)} / {train_size / (train_size + test_size)}')

Съотношение тренировъчни / тестови = 0.1780820153874057 / 0.8219179846125944


## Определяне на настроението на дадено ревю

- Слагаме по-голяма тежест върху заглавието на ревюто.
- Накрая нормализираме данните и ги вкарваме в интервала [1, 5]

In [395]:
def normalize_range(val, old_min, old_max):
	return (val - old_min) / (old_max - old_min)

def scale_value_to_range(value, input_range, output_range):
	(input_min, input_max) = input_range
	(output_min, output_max) = output_range

	value_factor = normalize_range(value, input_min, input_max);
	output_size = output_max - output_min;

	return output_min + output_size * value_factor

title_coeff = 5
input_range_boundary = (1 + title_coeff) / 2

def get_sentiment_for_review(title, review):
    title_score = get_sentiment_score(title)
    review_score = get_sentiment_score(review)
    
    return scale_value_to_range((title_score * title_coeff + review_score) / 2, (-input_range_boundary, input_range_boundary), (1, 5))

### Тест

In [396]:
print(f'Рейтинг: {get_sentiment_for_review("Worst phone ever!", "This phone is worthless. It has low battery capacity.")}')

Рейтинг: 1.9083333333333332


In [397]:
print(f'Рейтинг: {get_sentiment_for_review("Recommended!", "This book is fascinating!")}')

Рейтинг: 3.4791666666666665


## Изпълнение на алгоритъма върху целия корпус

In [412]:
def calc_accuracy(prediction, real_result):
    return 1 - abs(real_result - prediction) / real_result

def average(lst):
    return sum(lst) / len(lst)

accuracy_scores = []

all_data = pd.concat([train, test], ignore_index=True, sort=False)

for ind in range(2000):
    accuracy_scores.append(calc_accuracy(get_sentiment_for_review(all_data["title"][ind], all_data["review"][ind]), all_data["rating"][ind]))

print(f'Средна точност на предсказване: {average(accuracy_scores)}')


Средна точност на предсказване: 0.4310926015800473
